## BÀI TOÁN DỰ ĐOÁN CỔ PHIẾU LÊN HOẶC XUỐNG
Đầu vào: Các biến động của cổ phiếu của ngày hôm nay và các ngày trước đó.  
Đầu ra: Dự đoán cổ phiếu lên hoặc xuống

In [1]:
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import YearLocator
from hmm import viterbi, baum_welch

pd.options.mode.chained_assignment = None

In [2]:
MAX_STATE = 27 # maximum of observed state

### CÁC HÀM HỖ TRỢ

In [3]:
map_bit = {
    'L': 0,
    'M': 1,
    'H': 2
}

def convert(s: str):
    assert len(s) == 3
    val = 0
    for i in range(3):
        val += map_bit[s[i]] * (3**i)
    return val

def build_transition_matrix(df: pd.DataFrame):
    transition_matrix = np.zeros((2, 2), dtype=np.float32)
    for i in range(2):
        cnt = [0 for _ in range(2)]
        for idx in range(len(df)-1):
            if df['Outcome_Next_Day_Direction'][idx] == i:
                cnt[df['Outcome_Next_Day_Direction'][idx+1]] += 1 
        for j in range(2):
            transition_matrix[i][j] = cnt[j]/sum(cnt)
    return transition_matrix

def build_emission_matrix(df: pd.DataFrame):
    emission_matrix = np.zeros((2, MAX_STATE), dtype=np.float32)
    for j in range(2):
        cnt = [0 for _ in range(MAX_STATE)]
        for idx in range(len(df)):
            if df['Outcome_Next_Day_Direction'][idx] == j:
                cnt[df['Encode_Event'][idx]] += 1
        for i in range(MAX_STATE):
            emission_matrix[j, i] = cnt[i]/sum(cnt)
    return emission_matrix

### ĐỌC DỮ LIỆU
Dữ liệu được sử dụng trong bài này là dữ liệu cổ phiếu của công ty AMAZON từ ngày 03/01/2017 đến 31/12/2021.

In [4]:
df = pd.read_csv('data/AMZ_data.csv')
df.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2017-01-03,758.760010,747.700012,757.919983,753.669983,3521100,753.669983
1,2017-01-04,759.679993,754.200012,758.390015,757.179993,2510500,757.179993
2,2017-01-05,782.400024,760.260010,761.549988,780.450012,5830100,780.450012
3,2017-01-06,799.440002,778.479980,782.359985,795.989990,5986200,795.989990
4,2017-01-09,801.770020,791.770020,798.000000,796.919983,3446100,796.919983


### MÔ TẢ DỮ LIỆU
| Key                   | Definition    |
|-------                |--------       |
| **High** and **Low**  | Refer to the maximum and minimum prices in a given time period |
| **Open** and **Close**| The prices at which a stock began and ended trading in the same period |
| **Volume**            | The total amount of trading activity |
| **Adj Close**         | The stock's value after accounting for any corporate actions |

### XỬ LÝ DỮ LIỆU
- Tính toán độ chênh lệch giữa cổ phiếu giữa hai ngày (đơn vị là phần trăm). Các cột được thêm vào là `Close_gap`, `High_gap`, `Low_gap`, `Volume_gap`.
- Tính toán độ tăng lên của cổ phiếu (`Daily_change`) thông qua công thức
$$
    \dfrac{d_{Close} - d_{Open}}{d_{Open}}
$$
- Tính toán độ chênh lệch của cổ phiếu ngày hôm nay và ngày hôm sau (`Outcome_Next_Day_Direction`).
- Binning các độ chênh lệch theo phần trăm vào 3 giá trị là L (Low), M (Medium), H (High). Thêm các cột mới có hậu tố là `LMH`.
- Binning `Outcome_Next_Day_Direction` vào 2 giá trị là 1 nếu `Outcome_Next_Day_Direction` lớn hơn 0 và 0 nếu ngược lại
- Loại các cột `High_gap_LMH` và `Low_gap_LMH` do nó không có giá trị để dự đoán.
- Kết hợp các cột `Close_gap_LMH`, `Volume_gap_LMH` và `Daily_change_LMH` thành một là `Event_Pattern`.
- Encoding `Event_Pattern` (`Encode_Event`) thành các số để sau này dễ xử lý.

In [5]:
def process_df(df):
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    df['Close_gap'] = df['Close'].pct_change()
    df['High_gap'] = df['High'].pct_change()
    df['Low_gap'] = df['Low'].pct_change()
    df['Volume_gap'] = df['Volume'].pct_change()
    df['Daily_change'] = (df['Close'] - df['Open']) / df['Open']
    df['Outcome_Next_Day_Direction'] = (df['Volume'].shift(-1) - df['Volume'])
    df = df[1:-1]

    df['Close_gap_LMH'] = pd.qcut(df['Close_gap'], 3, labels=["L", "M", "H"])

    # High_Gap - not used in this example
    df['High_gap_LMH'] = pd.qcut(df['High_gap'], 3, labels=["L", "M", "H"])

    # Low_Gap - not used in this example
    df['Low_gap_LMH'] = pd.qcut(df['Low_gap'], 3, labels=["L", "M", "H"])

    # Volume_Gap
    df['Volume_gap_LMH'] = pd.qcut(df['Volume_gap'], 3, labels=["L", "M", "H"])
    
    # Daily_Change
    df['Daily_change_LMH'] = pd.qcut(df['Daily_change'], 3, labels=["L", "M", "H"])

    # compressed_set = df[abs(df['Outcome_Next_Day_Direction']) > 10000000]
    df['Outcome_Next_Day_Direction'] = np.where((df['Outcome_Next_Day_Direction'] > 0), 1, 0)

    df['Event_Pattern'] = df['Close_gap_LMH'].astype(str) + df['Volume_gap_LMH'].astype(str) + df['Daily_change_LMH'].astype(str)
    df = df[['Date', 'Event_Pattern', 'Outcome_Next_Day_Direction']]

    df['Encode_Event'] = df['Event_Pattern'].map(lambda x: convert(x))
    df = df.reset_index()
    return df

### ÁP DỤNG VÀO MÔ HÌNH HMM
Trạng thái có thể quan sát sẽ là `Encode_Event` (các biến động của cố phiếu của ngày hôm nay và các ngày hôm trước).  
Trạng thái ẩn là `Outcome_Next_Day_Direction` (cổ phiếu tăng hoặc giảm của ngày hôm sau).  

### CHIA TẬP HUẤN LUYỆN VÀ TẬP THỬ NGHIỆM
Tập thử nghiệm gồm 500 ngày cuối cùng. Tập huấn luyện gồm những ngày còn lại.

In [6]:
train_df = df[:-500]
val_df = df[-500:]

train_df = process_df(train_df)
val_df = process_df(val_df)

train_df.head(10)

,index,Date,Event_Pattern,Outcome_Next_Day_Direction,Encode_Event
0,1,2017-01-04,MLM,1,10
1,2,2017-01-05,HHH,1,26
2,3,2017-01-06,HMH,0,23
3,4,2017-01-09,MLM,0,10
4,5,2017-01-10,MLM,1,10
5,6,2017-01-11,MHH,1,25
6,7,2017-01-12,HHH,0,26
7,8,2017-01-13,MLM,0,10
8,9,2017-01-17,LML,0,3
9,10,2017-01-18,MLM,1,10


### XÂY DỰNG MA TRẬN TRANSITION VÀ EMISSION

In [7]:
A = build_transition_matrix(train_df)
B = build_emission_matrix(train_df)
initial_distribution = np.array((0.5, 0.5))

### CHẠY THUẬT TOÁN VITERBI VÀ TÍNH TOÁN ĐỘ CHÍNH XÁC

In [8]:
V = val_df['Encode_Event'].to_numpy()
pred = viterbi(V, A, B, initial_distribution)
label = val_df['Outcome_Next_Day_Direction'].to_numpy()

print(f'Acc: {(pred == label).sum() / len(label) * 100}%')

Acc: 65.26104417670683%


### NHẬN XÉT KẾT QUẢ
- Bài toán không sử dụng được thuật toán Baum Welch do khi độ dài chuỗi quan sát càng lớn, xác suất càng nhỏ dẫn tới xuất hiện giá trị NaN (vượt qua giới hạn trong ngôn ngữ Python).
- Kết quả trên còn khá thấp do một phần bài toán dự đoán cổ phiếu lên xuống còn dựa vào các yếu tố bên ngoài mà trong dữ liệu không có được.

### MỘT SỐ HƯỚNG CẢI TIẾN
- Sử dụng Rabiner Scaling cho HMM để có thể chạy được thuật toán Baum Welch với độ dài chuỗi quan sát lớn.
- Sử dụng các model ANN tiêu biểu cho bài toán dự đoán có chuỗi thời gian như là RNN với các lớp LSTM.